## Continuous Bag of Words (CBOW) 实现

In [1]:
from collections import defaultdict
import dynet as dy
import random
import time
import numpy as np

In [2]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))

In [3]:
UNK = w2i["<unk>"]

In [7]:
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [9]:
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [10]:
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [11]:
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [12]:
def calc_scores(words):
  dy.renew_cg()
  cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
  W_sm_exp = dy.parameter(W_sm)
  b_sm_exp = dy.parameter(b_sm)
  return W_sm_exp * cbow + b_sm_exp

In [13]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5395, time=0.30s
iter 0: test acc=0.3941
iter 1: train loss/sent=1.1529, time=0.29s
iter 1: test acc=0.3729
iter 2: train loss/sent=0.8755, time=0.30s
iter 2: test acc=0.3719
iter 3: train loss/sent=0.6427, time=0.31s
iter 3: test acc=0.3742
iter 4: train loss/sent=0.4709, time=0.30s
iter 4: test acc=0.3656
iter 5: train loss/sent=0.3351, time=0.34s
iter 5: test acc=0.3602
iter 6: train loss/sent=0.2438, time=0.34s
iter 6: test acc=0.3602
iter 7: train loss/sent=0.1782, time=0.34s
iter 7: test acc=0.3507
iter 8: train loss/sent=0.1303, time=0.32s
iter 8: test acc=0.3534
iter 9: train loss/sent=0.1025, time=0.34s
iter 9: test acc=0.3416
iter 10: train loss/sent=0.0790, time=0.31s
iter 10: test acc=0.3489
iter 11: train loss/sent=0.0591, time=0.30s
iter 11: test acc=0.3443
iter 12: train loss/sent=0.0467, time=0.29s
iter 12: test acc=0.3466
iter 13: train loss/sent=0.0363, time=0.32s
iter 13: test acc=0.3434
iter 14: train loss/sent=0.0331, time=0.31s
iter 14: t